In [3]:
from math import pi

import lenstronomy.Util.param_util as param_util
import torch
from astropy.cosmology import FlatLambdaCDM as FlatLambdaCDM_ap
from lenstronomy.LensModel.lens_model import LensModel

from caustic.cosmology import FlatLambdaCDM
from caustic.lenses import SIE, MultiplaneLens
from typing import Any, Dict, List, Union

import numpy as np
from lenstronomy.Data.pixel_grid import PixelGrid
from lenstronomy.LensModel.lens_model import LensModel

from caustic.lenses import ThinLens
from caustic.lenses.base import ThickLens
from caustic.utils import get_meshgrid

In [4]:
def lens_test_helper(
    lens: Union[ThinLens, ThickLens],
    lens_ls: LensModel,
    z_s,
    x,
    kwargs_ls: List[Dict[str, Any]],
    rtol,
    atol,
    test_alpha=True,
    test_Psi=True,
    test_kappa=True,
):
    if test_alpha:
        alpha_test_helper(lens, lens_ls, z_s, lens.pack(x), kwargs_ls, atol, rtol)

    if test_Psi:
        Psi_test_helper(lens, lens_ls, z_s, lens.pack(x), kwargs_ls, atol, rtol)

    if test_kappa:
        kappa_test_helper(lens, lens_ls, z_s, lens.pack(x), kwargs_ls, atol, rtol)


In [5]:
rtol = 0
atol = 5e-3

# Setup

z_s = torch.tensor(1.5, dtype=torch.float32)
cosmology = FlatLambdaCDM("cosmo")
cosmology.to(dtype=torch.float32)

# Parameters
xs = [
    [0.5, 0.9, -0.4, 0.9999, 3 * pi / 4, 0.8],
    [0.7, 0.0, 0.5, 0.9999, -pi / 6, 0.7],
    [1.1, 0.4, 0.3, 0.9999, pi / 4, 0.9],
]
x = torch.tensor([p for _xs in xs for p in _xs], dtype=torch.float32)


In [7]:
lens = MultiplaneLens(
    "multiplane", cosmology, [SIE(f"sie-{i}", cosmology) for i in range(len(xs))]
)

# lenstronomy
kwargs_ls = []
for _xs in xs:
    e1, e2 = param_util.phi_q2_ellipticity(phi=_xs[4], q=_xs[3])
    kwargs_ls.append(
        {
            "theta_E": _xs[5],
            "e1": e1,
            "e2": e2,
            "center_x": _xs[1],
            "center_y": _xs[2],
        }
    )


In [10]:
lens

MultiplaneLens(
    name='multiplane',
    static=[],
    dynamic=[],
    x keys=[('sie-0': ['z_l', 'thx0', 'thy0', 'q', 'phi', 'b']), ('sie-1': ['z_l', 'thx0', 'thy0', 'q', 'phi', 'b']), ('sie-2': ['z_l', 'thx0', 'thy0', 'q', 'phi', 'b'])]
)